# Megatron-Bridge Training Tutorial: Low-Precision Training

This tutorial demonstrates how to train large language models with different numerical precision formats (BF16, FP8, MXFP8, NVFP4) using Megatron-Bridge.

**What you'll learn:**
- Prepare and tokenize training datasets
- Train with BF16 (baseline), FP8, MXFP8, and NVFP4 precision
- Compare performance and memory usage across formats
- Switch precision formats with a single line of code

**Prerequisites:**
- 4 NVIDIA GB200 GPUs (Hopper can be used for BF16 and FP8)
- Megatron-Bridge installed
- HuggingFace account with Llama model access

**Container used:**
This tutorial uses the [NVIDIA NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo) from NGC, which includes Megatron-Bridge and all required dependencies.

## Step 1: Authenticate in HuggingFace

During the tutorial we need the tokenizer of the Llama 3.1 8B. You can get it from HuggingFace. For that, introduce your HuggingFace token and run the following command:

In [ ]:
# Authenticate with HuggingFace to access gated models (required for Llama)
# 
# Steps to get your token:
# 1. Go to https://huggingface.co/settings/tokens
# 2. Create a new token with "read" permissions
# 3. Accept the Llama model license at https://huggingface.co/meta-llama/Llama-3.1-8B
# 4. Replace MY_TOKEN below with your actual token
!hf auth login --token MY_TOKEN

## Step 2: Prepare Training Dataset

We'll download NVIDIA's Nemotron high-quality synthetic dataset and tokenize it for efficient training. Tokenization converts text to numerical token IDs and creates memory-mapped binary files for fast data loading during training.

In [ ]:
# Download NVIDIA Nemotron sample dataset and prepare train/test splits
# This high-quality synthetic dataset is designed for LLM pretraining
import json
import os
from datasets import load_dataset

def save_jsonl(dataset, jsonl_path, text_column="text"):
    """Save HuggingFace dataset to JSONL format (one JSON object per line)"""
    with open(jsonl_path, "w") as f:
        for sample in dataset:
            f.write(json.dumps({"text": sample[text_column]}) + "\n")

# Load NVIDIA Nemotron high-quality synthetic pretraining dataset
print("Loading dataset from HuggingFace...")
dataset = load_dataset("nvidia/Nemotron-Pretraining-Dataset-sample", "Nemotron-CC-High-Quality-Synthetic")
dataset = dataset["train"].shuffle()  # Shuffle for better training dynamics

# Split into 80% train, 20% test
print(f"Splitting dataset ({len(dataset)} samples total)...")
train_test = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test["train"]
test_dataset = train_test["test"]

# Save as JSONL files (required format for Megatron tokenization)
print("Saving datasets as JSONL files...")
os.makedirs("data/raw", exist_ok=True)
save_jsonl(train_dataset, "data/raw/train.jsonl")
save_jsonl(test_dataset, "data/raw/test.jsonl")
print(f"✓ Saved {len(train_dataset)} train and {len(test_dataset)} test samples")

In [ ]:
%%bash
# Tokenize text data using Megatron-LM's preprocessing tool
#
# Why tokenize ahead of time?
# - Converts text to token IDs once (not every epoch)
# - Creates memory-mapped binary files for fast random access
# - Enables efficient data loading during training
# - Saves CPU time during training
#
# Output files created:
# - train_text_document.bin (document/sequence data)
# - train_text_document.idx (document/sequence metadata)

MEGATRON_LM_PATH=/opt/megatron-lm/

echo "Tokenizing training data..."
python3 $MEGATRON_LM_PATH/tools/preprocess_data.py \
    --input ./data/raw/train.jsonl \
    --output-prefix ./data/tokenized/train \
    --tokenizer-type HuggingFaceTokenizer \
    --tokenizer-model meta-llama/Llama-3.1-8B \
    --log-interval 100 \
    --workers 4 \
    --append-eod  # Add end-of-document token after each sample

echo ""
echo "Tokenizing test data..."
python3 $MEGATRON_LM_PATH/tools/preprocess_data.py \
    --input ./data/raw/test.jsonl \
    --output-prefix ./data/tokenized/test \
    --tokenizer-type HuggingFaceTokenizer \
    --tokenizer-model meta-llama/Llama-3.1-8B \
    --log-interval 100 \
    --workers 4 \
    --append-eod

echo ""
echo "✓ Tokenization complete!"
ls -lh ./data/tokenized/

## Step 3: Explore the Default Configuration

Before we start training, let's examine the default Llama 3.1 8B configuration. This shows all the settings Megatron-Bridge uses, including model architecture, training hyperparameters, and precision settings. In the examples that follow, we'll only modify the `mixed_precision` parameter to compare different formats.

In [ ]:
# Load and examine the default Llama 3.1 8B pretraining configuration
# This config includes:
# - Model architecture (32 layers, 4096 hidden size, 32 attention heads, etc.)
# - Training hyperparameters (learning rate, batch size, optimizer settings)
# - Mixed precision settings (default: BF16)
# - Parallelism strategy (tensor, pipeline, data parallel)
# - Checkpoint and logging configuration
from pprint import pprint
from megatron.bridge.recipes.llama.llama3 import llama31_8b_pretrain_config as pretrain_config

print("Loading default Llama 3.1 8B configuration...")
cfg = pretrain_config()

print("\nConfiguration overview:")
print(f"  Model: Llama 3.1 8B")
print(f"  Default precision: {cfg.mixed_precision}")
print(f"  Training iterations: {cfg.train.train_iters}")
print(f"  Micro batch size: {cfg.train.micro_batch_size}")
print(f"  Global batch size: {cfg.train.global_batch_size}")
print("\nFull configuration:")
pprint(cfg)

## Understanding Numerical Precision Formats

**Mixed precision training** uses lower-precision number formats to accelerate training and reduce memory usage while maintaining model quality.

### Why Use Lower Precision?
- ⚡ **Faster computation**: Modern GPUs have specialized hardware for low-precision math
- 💾 **Less memory**: Smaller numbers = more data fits in GPU memory
- 🚀 **Larger models/batches**: Use saved memory for bigger models or batch sizes
- ✅ **Maintained quality**: Careful format design preserves training convergence

### Available Formats in Megatron-Bridge:
- **BF16** (16-bit): Industry standard, proven stability
- **FP8** (8-bit): Faster training on H100+, hybrid E4M3/E5M2 formats
- **MXFP8** (8-bit): Enhanced FP8 stability through microscaling
- **NVFP4** (4-bit): Maximum efficiency for Blackwell GPUs

This tutorial demonstrates how to switch between these formats with a single line of code!

## Training with Different Precision Formats

The configuration above shows all training settings. In the following examples, we'll modify only the `mixed_precision` parameter to compare BF16, FP8, MXFP8 and NVFP4 training while keeping everything else constant.

## Example 1: BF16 Training (Baseline)

**BF16** is the default format for training workloads. It offers:
- Proven numerical stability
- Faster than FP32 on modern GPUs
- No special tuning required

The training script below uses BF16 by default (no explicit `mixed_precision` setting needed).

In [ ]:
%%writefile llama_3.1_8b_default.py
from pprint import pprint
from megatron.bridge.recipes.llama.llama3 import llama31_8b_pretrain_config as pretrain_config
from megatron.bridge.training.gpt_step import forward_step
from megatron.bridge.training.pretrain import pretrain

if __name__ == "__main__":
    # Load default Llama 3.1 8B config with custom training parameters
    cfg = pretrain_config(
        train_iters=10,              # Number of training iterations
        micro_batch_size=2,          # Batch size per GPU
        global_batch_size=128,       # Total batch size across all GPUs
        lr_warmup_iters=10,          # Learning rate warmup iterations
        lr_decay_iters=20,           # Learning rate decay iterations
        context_parallel_size=1      # Context parallelism (1 = disabled)
    )
    
    # Configure dataset paths (train, validation, test)
    cfg.dataset.split = None
    cfg.dataset.blend_per_split = [
        [["./data/tokenized/train_text_document"], None],
        [["./data/tokenized/test_text_document"], None],
        [["./data/tokenized/test_text_document"], None]]
    
    # Logging and checkpoint settings
    cfg.logger.log_interval = 1         # Log every iteration
    cfg.checkpoint.save = None          # Don't save checkpoints (demo only)
    cfg.dataset.num_workers = 2         # Data loading workers
    cfg.train.eval_iters = 0            # Skip evaluation (faster demo)
    
    # the BF16 format is set by default
    # cfg.mixed_precision = "bf16_mixed"
    
    # Start training
    pretrain(cfg, forward_step)

In [ ]:
# Run BF16 training on 4 GPUs
!torchrun --standalone --nproc-per-node 4 llama_3.1_8b_default.py

With 4 GB200 you should see performance similar to following:
```
Step Time : 11.65s GPU utilization: 1158.1TFLOP/s/GPU
```

## Example 2: FP8 Training (H100+ Only)

**FP8** offers training speedup for the Hopper and Blackwell GPUs compared to BF16.

The script below adds one line to enable FP8: `cfg.mixed_precision = "bf16_with_fp8_current_scaling_mixed"`

This recipe uses:
- FP8 hybrid mode (E4M3 for weights and activations, E5M2 for gradients)
- BF16 for first and last transformer layers (stability)
- Current scaling (per-iteration scaling factors)

In [ ]:
%%writefile llama_3.1_8b_fp8.py
from pprint import pprint
from megatron.bridge.recipes.llama.llama3 import llama31_8b_pretrain_config as pretrain_config
from megatron.bridge.training.gpt_step import forward_step
from megatron.bridge.training.pretrain import pretrain

if __name__ == "__main__":
    # Same configuration as BF16 baseline
    cfg = pretrain_config(
        train_iters=10,
        micro_batch_size=2,
        global_batch_size=128,
        lr_warmup_iters=10,
        lr_decay_iters=20,
        context_parallel_size=1
    )
    
    # Dataset configuration (identical to BF16)
    cfg.dataset.split = None
    cfg.dataset.blend_per_split = [
        [["./data/tokenized/train_text_document"], None],
        [["./data/tokenized/test_text_document"], None],
        [["./data/tokenized/test_text_document"], None]]
    
    cfg.logger.log_interval = 1
    cfg.checkpoint.save = None
    cfg.dataset.num_workers = 2
    cfg.train.eval_iters = 0 

    # ⚡ ONLY CHANGE: Enable FP8 training (requires H100+ GPU)
    # This uses hybrid FP8: E4M3 for weights, E5M2 for gradients
    # with per-iteration (current) scaling for numerical stability
    cfg.mixed_precision = "bf16_with_fp8_current_scaling_mixed"
    
    pretrain(cfg, forward_step)

In [ ]:
# Run FP8 training on 4 GPUs (requires H100+)
!torchrun --standalone --nproc-per-node 4 llama_3.1_8b_fp8.py

With 4 GB200 you should see performance similar to following:
```
Step Time : 8.57s GPU utilization: 1574.0TFLOP/s/GPU
```

## Example 3: MXFP8 Training

**MXFP8** is available from the Blackwell generation of GPUs and offers comparable speedup to FP8 with enhanced stability.

The script below adds one line to enable FP8: `cfg.mixed_precision = "bf16_with_mxfp8_mixed"`

This recipe uses:
- FP8 E4M3 for weights, activations and gradients
- Scaling factors computed for each block of 32 elements
- Current scaling (per-iteration scaling factors)
- No layers need to be kept in BF16 for stability

In [ ]:
%%writefile llama_3.1_8b_mxfp8.py
from pprint import pprint
from megatron.bridge.recipes.llama.llama3 import llama31_8b_pretrain_config as pretrain_config
from megatron.bridge.training.gpt_step import forward_step
from megatron.bridge.training.pretrain import pretrain

if __name__ == "__main__":
    # Same configuration as BF16 baseline
    cfg = pretrain_config(
        train_iters=10,
        micro_batch_size=2,
        global_batch_size=128,
        lr_warmup_iters=10,
        lr_decay_iters=20,
        context_parallel_size=1
    )
    
    # Dataset configuration (identical to BF16)
    cfg.dataset.split = None
    cfg.dataset.blend_per_split = [
        [["./data/tokenized/train_text_document"], None],
        [["./data/tokenized/test_text_document"], None],
        [["./data/tokenized/test_text_document"], None]]
    
    cfg.logger.log_interval = 1
    cfg.checkpoint.save = None
    cfg.dataset.num_workers = 2
    cfg.train.eval_iters = 0
    
    # ⚡ ONLY CHANGE: Enable MXFP8 (Microscaling FP8) training
    # Uses block-based scaling (32 elements/block) for better stability
    # Optimized for Blackwell (B200+) GPUs
    cfg.mixed_precision = "bf16_with_mxfp8_mixed"
    
    pretrain(cfg, forward_step)

In [ ]:
# Run MXFP8 training on 4 GPUs
# Expected: ~8-9s per step, ~1500 TFLOP/s/GPU (~30% faster than BF16)
!torchrun --standalone --nproc-per-node 4 llama_3.1_8b_mxfp8.py

With 4 GB200 you should see performance similar to following:
```
Step Time : 8.63s GPU utilization: 1562.8TFLOP/s/GPU
```

## Example 4: NVFP4 Training

**NVFP4** is available from the Blackwell generation of GPUs and offers further speedups compared to FP8 and MXFP8 while maintaining accuracy.

The script below adds one line to enable NVFP4: `cfg.mixed_precision = "bf16_with_nvfp4_mixed()"`

This recipe uses:
- FP4 (E2M1) values with two-level scaling
- groups of 16 values share a high-precision FP8 (E4M3) scale plus a per-tensor FP32 scale
- further techniques such as Randomized Hadamard Transform, 2D block-scaling for the weights, and stochastic rounding for the gradients

We recommend to keep the last 15% of the layers in BF16 precision for better stability. As a result, for the Llama 3.1 8B model we keep 4 out of the 32 layers in BF16.

In [ ]:
%%writefile llama_3.1_8b_fp4.py
from pprint import pprint
from megatron.bridge.recipes.llama.llama3 import llama31_8b_pretrain_config as pretrain_config
from megatron.bridge.training.gpt_step import forward_step
from megatron.bridge.training.pretrain import pretrain

if __name__ == "__main__":
    # Same configuration as BF16 baseline
    cfg = pretrain_config(
        train_iters=10,
        micro_batch_size=2,
        global_batch_size=128,
        lr_warmup_iters=10,
        lr_decay_iters=20,
        context_parallel_size=1,
    )
    
    # Dataset configuration (identical to BF16)
    cfg.dataset.split = None
    cfg.dataset.blend_per_split = [
        [["./data/tokenized/train_text_document"], None],
        [["./data/tokenized/test_text_document"], None],
        [["./data/tokenized/test_text_document"], None]]
    
    cfg.logger.log_interval = 1
    cfg.checkpoint.save = None
    cfg.dataset.num_workers = 2
    cfg.train.eval_iters = 0
    
    # ⚠️ Enable NVFP4 training keeping the last 4 layers in BF16
    from megatron.bridge.training.mixed_precision import MixedPrecisionConfig, bf16_with_nvfp4_mixed
    def bf16_with_nvfp4_mixed_4_layers_bf16() -> MixedPrecisionConfig:
        cfg = bf16_with_nvfp4_mixed()
        cfg.first_last_layers_bf16 = True
        cfg.num_layers_at_end_in_bf16 = 4
        return cfg
    cfg.mixed_precision = bf16_with_nvfp4_mixed_4_layers_bf16()
    pretrain(cfg, forward_step)


In [ ]:
# Run NVFP4 training on 4 GPUs
!torchrun --standalone --nproc-per-node 4 llama_3.1_8b_fp4.py

With 4 GB200 you should see performance similar to following:
```
Step Time : 7.47s GPU utilization: 1805.6TFLOP/s/GPU
```

# Summary

## Quick Guide

**Older GPUs:** BF16 (default)
```python
cfg.mixed_precision = "bf16_mixed"  # or omit
```

**H100+, higher throughput:** FP8
```python
cfg.mixed_precision = "bf16_with_fp8_current_scaling_mixed"
```

**B200+, better stability:** MXFP8
```python
cfg.mixed_precision = "bf16_with_mxfp8_mixed"
```

**B200+, highest throughput:** NVFP4
```python
cfg.mixed_precision = "bf16_with_nvfp4_mixed"
```

## Key Takeaways

1. **Start with BF16** - establish a stable baseline first
2. **Validate with lower precision** - Compare convergence, loss curves, final model quality
3. **Hardware matters** - FP8 beneficial on H100+, MXFP8/NVFP4 on B200+
4. **One line config** - Simple to switch: `cfg.mixed_precision = "recipe_name"`
5. **Memory savings** - Use lower precision to increase batch size or model size


## Expected Performance (Llama 3.1 8B on 4x GB200 GPUs)

| Precision  | Step Time | GPU Utilization  | Speedup vs BF16 | Hardware |
|------------|-----------|------------------|-----------------|----------|
| **BF16**   | ~11.65s   | 1158 TFLOP/s/GPU |         -       |  Any GPU |
| **FP8**    | ~8.57s    | 1574 TFLOP/s/GPU |       1.36x     |   H100+  |
| **MXFP8**  | ~8.63s    | 1562 TFLOP/s/GPU |       1.35x     |   B200+  |
| **NVFP4**  | ~7.47s    | 1805 TFLOP/s/GPU |       1.56x     |   B200+  |

*Note: Performance numbers are approximate and will vary based on model size, batch size, and hardware configuration.*

## Resources

- [Pretraining Large Language Models with NVFP4](https://arxiv.org/abs/2509.25149)
- [NVFP4 blogpost](https://developer.nvidia.com/blog/introducing-nvfp4-for-efficient-and-accurate-low-precision-inference/
)
- [Mixed Precision Training Guide](https://docs.nvidia.com/nemo/megatron-bridge/latest/training/mixed-precision.html)
- [Using FP8 and FP4 with Transformer Engine](https://docs.nvidia.com/deeplearning/transformer-engine/user-guide/examples/fp8_primer.html)

Happy training! 🚀
